# OpenFOAM HRM Log Analysis

This program extracts data from HRMFoam log files for analysis (i.e. net mass flux).
    
    @author Daniel Duke <daniel.duke@monash.edu>
    @copyright (c) 2020 LTRAC
    @license GPL-3.0+
    @version 0.0.1
    @date 02/11/2022
        __   ____________    ___    ______
       / /  /_  ____ __  \  /   |  / ____/
      / /    / /   / /_/ / / /| | / /
     / /___ / /   / _, _/ / ___ |/ /_________
    /_____//_/   /_/ |__\/_/  |_|\__________/

    Laboratory for Turbulence Research in Aerospace & Combustion (LTRAC)
    Monash University, Australia


In [1]:
# Load modules
import numpy as np
import glob, copy, gzip, natsort, tqdm, os
import h5py

In [2]:
# Read HRMFoam Log file, and extract key parameters in log at each iteration such as mass fluxes.
def read_logfile(logFile):
    time = []
    data = {}
    unit = {}
    
    print("Scanning "+logFile+"...")
    
    with gzip.open(logFile,'r') as F:
        Nlines=len(F.readlines())
    
    with gzip.open(logFile,'r') as F:
        print("Reading data...")
        pbar = tqdm.tqdm(total=Nlines)
        
        l=F.readline(); n=1; m=0
        while n<Nlines:
            
            if (not b'Time =' in l) or (b'ClockTime' in l):
                # keep looping until we see 'Time =' string
                # make exception for the ExecutionTime/ClockTime string at end of run - ignore it
                l=F.readline(); n+=1
                
            else:
                # Begin iteration block
                pbar.update(n-m); m=n
                # Record the time for this block
                time.append(float(l.decode('ascii').split('=')[1]))
                # Next line
                l=F.readline(); n+=1

                # Keep going until the next 'Time =' string indicating a new iteration
                # Make an exception for the 'Net mass flux' string which has 'Time' in it
                while (not b'Time =' in l) or (b'Net mass flux' in l):

                    # Pressure/velocity data string
                    # Make exception for some error codes
                    if (b' is ' in l) and (not b'First token' in l) and (not b'Check header' in l):
                        # Break apart this string by the word 'is' and the units brackets
                        s = l.decode('ascii').strip().replace('is','[').replace(']','[').split('[')
                        # remove leading and trailing whitespace
                        s = [ ss.strip() for ss in s ]
                        # Break apart each variable into name, value and units
                        for i in range(0,len(s)-2,3):
                            if s[i] in data.keys(): # not first time
                                data[s[i]].append(float(s[i+1]))
                            else: # first time
                                data[s[i]]=[float(s[i+1])]
                                unit[s[i]]=s[i+2]

                    # Mass fluxes data string
                    elif b' at ' in l:
                        # Break apart this string by the = sign
                        s = l.decode('ascii').strip().split('=')
                        # remove leading and trailing whitespace
                        s = [ ss.strip() for ss in s ]
                        if(len(s)>1):
                            if s[0] in data.keys(): # not first time
                                data[s[0]].append(float(s[1]))
                            else:
                                data[s[0]]=[float(s[1])]
                                unit[s[0]]=''

                    elif b'Net mass flux' in l:
                         # Break apart this string by the = sign
                        s = l.decode('ascii').strip().split('=')
                        # remove leading and trailing whitespace
                        s = [ ss.strip() for ss in s ]
                        if 'Net mass flux' in data.keys(): # not first time
                            data['Net mass flux'].append(float(s[-1]))
                        else:
                            data['Net mass flux']=[float(s[-1])]
                            unit['Net mass flux']=''
                    
                    elif n>=Nlines: break # detect premature EoF
                    
                    l=F.readline(); n+=1
                    
            # end iteration block
            
    pbar.close()
    
    # Convert data to NumPy Arrays
    for k in data.keys():
        data[k] = np.array(data[k])
    time=np.array(time)
    
    # Remove empty keys
    if '' in data.keys():
        del data['']
    
    return time, data, unit

## Main program
Read logs and write them to a HDF5 file.

In [3]:
def readCase(logFiles, outputFile):

    # Read all log files and write to HDF5
    with h5py.File(outputFile,'w') as H:
        for logFile in natsort.natsorted(glob.glob(logFiles)):

            time,data,unit = read_logfile(logFile)

            G=H.create_group(os.path.basename(logFile))
            t_=G.create_dataset('time',data=time,compression='gzip')
            t_.attrs['unit']='s'
            for k in data.keys():
                try:
                    d_=G.create_dataset(k,data=data[k],compression='gzip')
                    d_.attrs['unit']=unit[k]
                except:
                    print(data.keys())
                    raise

            print("Wrote %i variables over %i iterations to %s" % (len(data.keys()),len(time),G.name))
            print("")

## Call main program once.

In [ ]:
# One run
topLevel = "/mnt/internal/2021_pmdi/newGeomTrial/postProcessing/massFlux/convergence_134a15pcEtOH/"
case = "ures"
logFiles = topLevel + case + "/*"
outputFile = topLevel + case + ".h5"
readCase(logFiles, outputFile)

In [ ]:
# One run
topLevel = "/mnt/internal/2021_pmdi/newGeomTrial/postProcessing/massFlux/ures/"
case = "152a15pcEtOH"
logFiles = topLevel + case + "/*"
outputFile = topLevel + case + ".h5"
readCase(logFiles, outputFile)

### Variable Geometry

In [4]:
topLevel = "/mnt/internal-hdd/2021_pmdi/newGeomTrial/postProcessing/massFlux/newGeomTrial/"
for p in ['134a','152a','1234ze','134a15pcEtOH','152a15pcEtOH','1234ze15pcEtOH']:
    
    logFiles = topLevel + "noz?_" + p + ".txt.gz"
        
    outputFile = topLevel + "newGeomTrial_"  + p + ".h5"
    
    if not os.path.isfile(outputFile):
        print("-- Writing into "+outputFile)
        readCase(logFiles, outputFile)
    else:
        print("-- Skipped existing "+outputFile)
    
print("Finished.")

-- Skipped existing /mnt/internal-hdd/2021_pmdi/newGeomTrial/postProcessing/massFlux/newGeomTrial/newGeomTrial_134a.h5
-- Writing into /mnt/internal-hdd/2021_pmdi/newGeomTrial/postProcessing/massFlux/newGeomTrial/newGeomTrial_152a.h5
Scanning /mnt/internal-hdd/2021_pmdi/newGeomTrial/postProcessing/massFlux/newGeomTrial/noz1_152a.txt.gz...
Reading data...


100%|██████████████████████████████████████████████████████████████████████████████████▉| 44144839/44144879 [02:07<00:00, 345025.15it/s]


Wrote 14 variables over 1036081 iterations to /noz1_152a.txt.gz

Scanning /mnt/internal-hdd/2021_pmdi/newGeomTrial/postProcessing/massFlux/newGeomTrial/noz2_152a.txt.gz...
Reading data...


100%|████████████████████████████████████████████████████████████████████████████████▉| 121208024/121208048 [05:07<00:00, 394523.84it/s]


Wrote 14 variables over 2884317 iterations to /noz2_152a.txt.gz

Scanning /mnt/internal-hdd/2021_pmdi/newGeomTrial/postProcessing/massFlux/newGeomTrial/noz3_152a.txt.gz...
Reading data...


...

0it [00:00, ?it/s]

Wrote 0 variables over 0 iterations to /noz8_1234ze.txt.gz

Scanning /mnt/internal-hdd/2021_pmdi/newGeomTrial/postProcessing/massFlux/newGeomTrial/noz9_1234ze.txt.gz...


Reading data...


100%|████████████████████████████████████████████████████████████████████████████████▉| 142916493/142916506 [06:49<00:00, 349409.88it/s]


Wrote 14 variables over 3359193 iterations to /noz9_1234ze.txt.gz

-- Skipped existing /mnt/internal-hdd/2021_pmdi/newGeomTrial/postProcessing/massFlux/newGeomTrial/newGeomTrial_134a15pcEtOH.h5
-- Skipped existing /mnt/internal-hdd/2021_pmdi/newGeomTrial/postProcessing/massFlux/newGeomTrial/newGeomTrial_152a15pcEtOH.h5
-- Skipped existing /mnt/internal-hdd/2021_pmdi/newGeomTrial/postProcessing/massFlux/newGeomTrial/newGeomTrial_1234ze15pcEtOH.h5
Finished.
